## Continue ensembles from v1 on full dataset format commonly used on Kaggle

### Integrated mlflow lets this notebook rinse-repeat

In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import Counter
import mlflow

# local imports
from prepare import *
from evaluate import *

### Set new experiment to keep track in mlflow

In [2]:
mlflow.set_experiment('drop_counts_higher_6k')

INFO: 'drop_counts_higher_6k' does not exist. Creating a new experiment


### Read in initial datasets if needed

In [3]:
#raw_train, raw_train_labels, raw_test, specs, sample = read_raw_csvs()
#raw_train_labels = pd.read_csv('data/train_labels.csv')

### Load large train/test features from Josh's work

In [4]:
reduced_train = pd.read_csv('reduce_train_cc_6k.csv')
reduced_test = pd.read_csv('reduce_test_cc_6k.csv')
reduced_train.shape, reduced_test.shape

((12906, 885), (972, 885))

### Start throwing model mud at the wall

In [5]:
from catboost import CatBoostClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier,
    BaggingClassifier,
    GradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
import warnings

warnings.filterwarnings(
    "ignore"
)  # Ridge classifier throws some warnings about ill-conditioned matrix

C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\imp.py:343: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return _load(spec)


### Baseline accuracy of 50% (or 25% if balanced)

In [6]:
reduced_train.accuracy_group.value_counts(normalize=True)

3    0.522160
0    0.222067
1    0.131102
2    0.124671
Name: accuracy_group, dtype: float64

# Initialize Models and start testing accuracy

In [7]:
rf = RandomForestClassifier()
rc = RidgeClassifier()
ac = AdaBoostClassifier()
et = ExtraTreesClassifier()
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()

clf = CatBoostClassifier(
    loss_function='MultiClass',
    task_type="CPU",
    learning_rate=0.05,
    iterations=2000,
    od_type="Iter",
    early_stopping_rounds=500,
    random_seed=42
    )

# tried with minimal results
#nb = GaussianNB()
#lr = LogisticRegression()               ## if they are commented out, they weren't performing well (or operator error...)
#sgd = SGDClassifier()

In [8]:
knn = KNeighborsClassifier()
svc = SVC(probability=True, verbose=1)

## Evaluate model performance

In [9]:
# Catboost - accuracy 56%a
clf = quick_eval(reduced_train, clf)#, cv=True)

0:	learn: 1.3493417	total: 142ms	remaining: 4m 43s
1:	learn: 1.3183031	total: 209ms	remaining: 3m 28s
2:	learn: 1.2910749	total: 277ms	remaining: 3m 4s
3:	learn: 1.2678645	total: 342ms	remaining: 2m 50s
4:	learn: 1.2466988	total: 420ms	remaining: 2m 47s
5:	learn: 1.2283583	total: 500ms	remaining: 2m 46s
6:	learn: 1.2084630	total: 578ms	remaining: 2m 44s
7:	learn: 1.1928979	total: 650ms	remaining: 2m 41s
8:	learn: 1.1785487	total: 724ms	remaining: 2m 40s
9:	learn: 1.1642233	total: 803ms	remaining: 2m 39s
10:	learn: 1.1525035	total: 884ms	remaining: 2m 39s
11:	learn: 1.1418792	total: 958ms	remaining: 2m 38s
12:	learn: 1.1329387	total: 1.04s	remaining: 2m 38s
13:	learn: 1.1241670	total: 1.13s	remaining: 2m 39s
14:	learn: 1.1151452	total: 1.21s	remaining: 2m 40s
15:	learn: 1.1084063	total: 1.3s	remaining: 2m 41s
16:	learn: 1.1001990	total: 1.38s	remaining: 2m 41s
17:	learn: 1.0940283	total: 1.46s	remaining: 2m 40s
18:	learn: 1.0876109	total: 1.53s	remaining: 2m 39s
19:	learn: 1.0820868	tot

160:	learn: 0.9460001	total: 10.5s	remaining: 2m
161:	learn: 0.9456937	total: 10.6s	remaining: 2m
162:	learn: 0.9454210	total: 10.6s	remaining: 1m 59s
163:	learn: 0.9451783	total: 10.7s	remaining: 1m 59s
164:	learn: 0.9450028	total: 10.8s	remaining: 1m 59s
165:	learn: 0.9444489	total: 10.8s	remaining: 1m 59s
166:	learn: 0.9438466	total: 10.9s	remaining: 1m 59s
167:	learn: 0.9436035	total: 10.9s	remaining: 1m 59s
168:	learn: 0.9429903	total: 11s	remaining: 1m 59s
169:	learn: 0.9423100	total: 11.1s	remaining: 1m 59s
170:	learn: 0.9419181	total: 11.1s	remaining: 1m 59s
171:	learn: 0.9410999	total: 11.2s	remaining: 1m 59s
172:	learn: 0.9404236	total: 11.3s	remaining: 1m 59s
173:	learn: 0.9400419	total: 11.3s	remaining: 1m 58s
174:	learn: 0.9394745	total: 11.4s	remaining: 1m 58s
175:	learn: 0.9392100	total: 11.4s	remaining: 1m 58s
176:	learn: 0.9390086	total: 11.5s	remaining: 1m 58s
177:	learn: 0.9383145	total: 11.6s	remaining: 1m 58s
178:	learn: 0.9378530	total: 11.6s	remaining: 1m 58s
179

316:	learn: 0.8860729	total: 21.4s	remaining: 1m 53s
317:	learn: 0.8855166	total: 21.5s	remaining: 1m 53s
318:	learn: 0.8853814	total: 21.6s	remaining: 1m 53s
319:	learn: 0.8849806	total: 21.6s	remaining: 1m 53s
320:	learn: 0.8846505	total: 21.7s	remaining: 1m 53s
321:	learn: 0.8842368	total: 21.8s	remaining: 1m 53s
322:	learn: 0.8840585	total: 21.8s	remaining: 1m 53s
323:	learn: 0.8839256	total: 21.9s	remaining: 1m 53s
324:	learn: 0.8834360	total: 22s	remaining: 1m 53s
325:	learn: 0.8829720	total: 22.1s	remaining: 1m 53s
326:	learn: 0.8826020	total: 22.1s	remaining: 1m 53s
327:	learn: 0.8822454	total: 22.2s	remaining: 1m 53s
328:	learn: 0.8820906	total: 22.3s	remaining: 1m 53s
329:	learn: 0.8817083	total: 22.3s	remaining: 1m 53s
330:	learn: 0.8812993	total: 22.4s	remaining: 1m 52s
331:	learn: 0.8810502	total: 22.5s	remaining: 1m 52s
332:	learn: 0.8807319	total: 22.6s	remaining: 1m 52s
333:	learn: 0.8803568	total: 22.6s	remaining: 1m 52s
334:	learn: 0.8802276	total: 22.7s	remaining: 1m

472:	learn: 0.8442932	total: 32.4s	remaining: 1m 44s
473:	learn: 0.8441015	total: 32.4s	remaining: 1m 44s
474:	learn: 0.8439843	total: 32.5s	remaining: 1m 44s
475:	learn: 0.8434984	total: 32.6s	remaining: 1m 44s
476:	learn: 0.8433355	total: 32.6s	remaining: 1m 44s
477:	learn: 0.8432830	total: 32.7s	remaining: 1m 44s
478:	learn: 0.8429550	total: 32.8s	remaining: 1m 44s
479:	learn: 0.8427710	total: 32.8s	remaining: 1m 43s
480:	learn: 0.8424959	total: 32.9s	remaining: 1m 43s
481:	learn: 0.8424248	total: 33s	remaining: 1m 43s
482:	learn: 0.8422704	total: 33s	remaining: 1m 43s
483:	learn: 0.8420627	total: 33.1s	remaining: 1m 43s
484:	learn: 0.8419258	total: 33.1s	remaining: 1m 43s
485:	learn: 0.8417865	total: 33.2s	remaining: 1m 43s
486:	learn: 0.8412493	total: 33.3s	remaining: 1m 43s
487:	learn: 0.8410435	total: 33.3s	remaining: 1m 43s
488:	learn: 0.8408267	total: 33.4s	remaining: 1m 43s
489:	learn: 0.8407025	total: 33.5s	remaining: 1m 43s
490:	learn: 0.8402333	total: 33.5s	remaining: 1m 4

629:	learn: 0.8103352	total: 44.7s	remaining: 1m 37s
630:	learn: 0.8102354	total: 44.8s	remaining: 1m 37s
631:	learn: 0.8101377	total: 44.9s	remaining: 1m 37s
632:	learn: 0.8099403	total: 45s	remaining: 1m 37s
633:	learn: 0.8098426	total: 45s	remaining: 1m 37s
634:	learn: 0.8097930	total: 45.1s	remaining: 1m 36s
635:	learn: 0.8097325	total: 45.2s	remaining: 1m 36s
636:	learn: 0.8093813	total: 45.3s	remaining: 1m 36s
637:	learn: 0.8090666	total: 45.3s	remaining: 1m 36s
638:	learn: 0.8087338	total: 45.4s	remaining: 1m 36s
639:	learn: 0.8085772	total: 45.5s	remaining: 1m 36s
640:	learn: 0.8083717	total: 45.6s	remaining: 1m 36s
641:	learn: 0.8083508	total: 45.6s	remaining: 1m 36s
642:	learn: 0.8082902	total: 45.7s	remaining: 1m 36s
643:	learn: 0.8081082	total: 45.8s	remaining: 1m 36s
644:	learn: 0.8079190	total: 45.8s	remaining: 1m 36s
645:	learn: 0.8076911	total: 45.9s	remaining: 1m 36s
646:	learn: 0.8076697	total: 45.9s	remaining: 1m 36s
647:	learn: 0.8074349	total: 46s	remaining: 1m 36s

...

1249:	learn: 0.6994245	total: 1m 37s	remaining: 58.3s
1250:	learn: 0.6992538	total: 1m 37s	remaining: 58.2s
1251:	learn: 0.6991402	total: 1m 37s	remaining: 58.1s
1252:	learn: 0.6989907	total: 1m 37s	remaining: 58.1s
1253:	learn: 0.6986926	total: 1m 37s	remaining: 58s
1254:	learn: 0.6985657	total: 1m 37s	remaining: 57.9s
1255:	learn: 0.6984749	total: 1m 37s	remaining: 57.9s
1256:	learn: 0.6984080	total: 1m 37s	remaining: 57.8s
1257:	learn: 0.6983023	total: 1m 37s	remaining: 57.7s
1258:	learn: 0.6981732	total: 1m 37s	remaining: 57.6s
1259:	learn: 0.6980547	total: 1m 37s	remaining: 57.5s
1260:	learn: 0.6979449	total: 1m 38s	remaining: 57.5s
1261:	learn: 0.6977914	total: 1m 38s	remaining: 57.4s
1262:	learn: 0.6975718	total: 1m 38s	remaining: 57.3s
1263:	learn: 0.6974070	total: 1m 38s	remaining: 57.2s
1264:	learn: 0.6972782	total: 1m 38s	remaining: 57.1s
1265:	learn: 0.6969613	total: 1m 38s	remaining: 57.1s
1266:	learn: 0.6967269	total: 1m 38s	remaining: 57s
1267:	learn: 0.6965986	total: 1m

1402:	learn: 0.6778964	total: 1m 49s	remaining: 46.7s
1403:	learn: 0.6777509	total: 1m 49s	remaining: 46.7s
1404:	learn: 0.6776936	total: 1m 50s	remaining: 46.6s
1405:	learn: 0.6773815	total: 1m 50s	remaining: 46.5s
1406:	learn: 0.6772713	total: 1m 50s	remaining: 46.5s
1407:	learn: 0.6771813	total: 1m 50s	remaining: 46.4s
1408:	learn: 0.6770766	total: 1m 50s	remaining: 46.3s
1409:	learn: 0.6768559	total: 1m 50s	remaining: 46.3s
1410:	learn: 0.6766528	total: 1m 50s	remaining: 46.2s
1411:	learn: 0.6765401	total: 1m 50s	remaining: 46.1s
1412:	learn: 0.6764610	total: 1m 50s	remaining: 46.1s
1413:	learn: 0.6763459	total: 1m 51s	remaining: 46s
1414:	learn: 0.6761777	total: 1m 51s	remaining: 46s
1415:	learn: 0.6759905	total: 1m 51s	remaining: 45.9s
1416:	learn: 0.6758426	total: 1m 51s	remaining: 45.8s
1417:	learn: 0.6755977	total: 1m 51s	remaining: 45.8s
1418:	learn: 0.6754311	total: 1m 51s	remaining: 45.7s
1419:	learn: 0.6753312	total: 1m 51s	remaining: 45.7s
1420:	learn: 0.6751895	total: 1m

1557:	learn: 0.6566406	total: 2m 4s	remaining: 35.2s
1558:	learn: 0.6565689	total: 2m 4s	remaining: 35.2s
1559:	learn: 0.6564869	total: 2m 4s	remaining: 35.1s
1560:	learn: 0.6564411	total: 2m 4s	remaining: 35s
1561:	learn: 0.6563894	total: 2m 4s	remaining: 34.9s
1562:	learn: 0.6562056	total: 2m 4s	remaining: 34.8s
1563:	learn: 0.6561624	total: 2m 4s	remaining: 34.8s
1564:	learn: 0.6559806	total: 2m 4s	remaining: 34.7s
1565:	learn: 0.6559311	total: 2m 4s	remaining: 34.6s
1566:	learn: 0.6558548	total: 2m 4s	remaining: 34.5s
1567:	learn: 0.6556669	total: 2m 4s	remaining: 34.4s
1568:	learn: 0.6555541	total: 2m 5s	remaining: 34.4s
1569:	learn: 0.6553931	total: 2m 5s	remaining: 34.3s
1570:	learn: 0.6550835	total: 2m 5s	remaining: 34.2s
1571:	learn: 0.6549074	total: 2m 5s	remaining: 34.1s
1572:	learn: 0.6547721	total: 2m 5s	remaining: 34s
1573:	learn: 0.6545885	total: 2m 5s	remaining: 34s
1574:	learn: 0.6543795	total: 2m 5s	remaining: 33.9s
1575:	learn: 0.6542796	total: 2m 5s	remaining: 33.8s

1713:	learn: 0.6363448	total: 2m 17s	remaining: 22.9s
1714:	learn: 0.6362889	total: 2m 17s	remaining: 22.8s
1715:	learn: 0.6361958	total: 2m 17s	remaining: 22.7s
1716:	learn: 0.6361323	total: 2m 17s	remaining: 22.7s
1717:	learn: 0.6360173	total: 2m 17s	remaining: 22.6s
1718:	learn: 0.6359100	total: 2m 17s	remaining: 22.5s
1719:	learn: 0.6358518	total: 2m 17s	remaining: 22.4s
1720:	learn: 0.6357315	total: 2m 17s	remaining: 22.3s
1721:	learn: 0.6355801	total: 2m 17s	remaining: 22.3s
1722:	learn: 0.6354971	total: 2m 17s	remaining: 22.2s
1723:	learn: 0.6354597	total: 2m 18s	remaining: 22.1s
1724:	learn: 0.6353674	total: 2m 18s	remaining: 22s
1725:	learn: 0.6352151	total: 2m 18s	remaining: 21.9s
1726:	learn: 0.6350801	total: 2m 18s	remaining: 21.9s
1727:	learn: 0.6349392	total: 2m 18s	remaining: 21.8s
1728:	learn: 0.6347975	total: 2m 18s	remaining: 21.7s
1729:	learn: 0.6347090	total: 2m 18s	remaining: 21.6s
1730:	learn: 0.6346632	total: 2m 18s	remaining: 21.5s
1731:	learn: 0.6345195	total: 

1868:	learn: 0.6172514	total: 2m 28s	remaining: 10.4s
1869:	learn: 0.6172111	total: 2m 28s	remaining: 10.4s
1870:	learn: 0.6170227	total: 2m 28s	remaining: 10.3s
1871:	learn: 0.6169477	total: 2m 29s	remaining: 10.2s
1872:	learn: 0.6168528	total: 2m 29s	remaining: 10.1s
1873:	learn: 0.6168061	total: 2m 29s	remaining: 10s
1874:	learn: 0.6167104	total: 2m 29s	remaining: 9.95s
1875:	learn: 0.6165906	total: 2m 29s	remaining: 9.87s
1876:	learn: 0.6165080	total: 2m 29s	remaining: 9.79s
1877:	learn: 0.6163311	total: 2m 29s	remaining: 9.71s
1878:	learn: 0.6162189	total: 2m 29s	remaining: 9.63s
1879:	learn: 0.6160979	total: 2m 29s	remaining: 9.55s
1880:	learn: 0.6160300	total: 2m 29s	remaining: 9.47s
1881:	learn: 0.6158607	total: 2m 29s	remaining: 9.39s
1882:	learn: 0.6158015	total: 2m 29s	remaining: 9.31s
1883:	learn: 0.6155906	total: 2m 29s	remaining: 9.23s
1884:	learn: 0.6154956	total: 2m 29s	remaining: 9.15s
1885:	learn: 0.6154008	total: 2m 30s	remaining: 9.07s
1886:	learn: 0.6151767	total: 

In [10]:
rf = quick_eval(reduced_train, rf, cv=True)  #.563cv acc
rc = quick_eval(reduced_train, rc, cv=True)  #.528cv acc
ac = quick_eval(reduced_train, ac, cv=True)  #.529cv acc
et = quick_eval(reduced_train, et, cv=True)  #.5399cv acc
bc = quick_eval(reduced_train, bc, cv=True)  #.511cv acc
gbc = quick_eval(reduced_train, gbc, cv=True)  #.435 cv acc



The CV qwk score of RandomForestClassifier is 0.38635117690353693
The CV qwk score of RidgeClassifier is 0.3734566961040053
The CV qwk score of AdaBoostClassifier is 0.296944266275207
The CV qwk score of ExtraTreesClassifier is 0.3321353873156597
The CV qwk score of BaggingClassifier is 0.4180782743126879
The CV qwk score of GradientBoostingClassifier is 0.32725231698357893


In [11]:
#quick_eval(reduced_train, mlp, cv=True)   # 0.295
#quick_eval(reduced_train, gpc) #   .498
#quick_eval(reduced_train, rbf, cv=True)   poor
#quick_eval(reduced_train, dt, cv=True)     #.411

In [ ]:

bc_rf_5 = BaggingClassifier(
    base_estimator=RandomForestClassifier(max_depth=5),
    n_estimators=20)#The accuracy of BaggingClassifier is 0.5740531373657434
                    #The QWK of BaggingClassifier is 0.40494994168142406

bc_gbc = BaggingClassifier(
    base_estimator=GradientBoostingClassifier(),
    n_estimators=20)
# The accuracy of BaggingClassifier is 0.6237987563595252
# The QWK of BaggingClassifier is 0.5252575839341429

bc_abc = BaggingClassifier(
    base_estimator=AdaBoostClassifier(),
    n_estimators=20)
# The accuracy of BaggingClassifier is 0.610231769361221
# The QWK of BaggingClassifier is 0.4936903908658856

bc_rf_5 = quick_eval(reduced_train, bc_rf_5)
# quick_eval(reduced_train, bc_rf_10)    # 0.549 with 20 estimators max depth of 10  (.377 with balanced)    0.551 with 200 
# quick_eval(reduced_train, bc_rf_50) 
bc_gbc = quick_eval(reduced_train, bc_gbc)   # 0.576 with 20 estimators (.389 with balanced)                     0.578 with 200
bc_abc = quick_eval(reduced_train, bc_abc)   # 0.563 with 20 estimators  (.374 with balanced)                    0.561 with 200

The accuracy of BaggingClassifier is 0.5530596436870643
The QWK of BaggingClassifier is 0.18672580579927922


## Model Ensembling

In [ ]:
estimators = [
    ('Adaboost',ac),
    ('rf', rf), 
    ('gbc', gbc),
    ('et', et),

    ('rc', rc),

    ('bc_rf_5', bc_rf_5),

    ('bc_gbc', bc_gbc),
    ('bc_abc', bc_abc),
]

stacking_clf = StackingClassifier(estimators=estimators, 
                final_estimator=RandomForestClassifier(), n_jobs=-1)        # created, load from pickle
quick_eval(reduced_train, stacking_clf)

In [ ]:
vc = VotingClassifier(estimators=estimators,
     n_jobs=-1,
     voting='hard')

quick_eval(reduced_train, vc)#, cv=True)     #initial w/o catboost .564